In [1]:
import os 
os.chdir('/work/jh2lab')

In [2]:
os.getcwd()

'/work/jh2lab'

In [8]:
# make test df

## get time info dataframe from textgrid
import re
import pandas as pd
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch


def get_audio_df(textgrid_dir):
    
    #with open(textgrid_dir, 'r', encoding='utf-16') as file:
    #with open(textgrid_dir, 'r', encoding='utf-8') as file:
        lines = [line.rstrip() for line in file]
    try:
        with open(file, 'r', encoding='utf-8') as f:
            lines = [line.rstrip() for line in f]
    except:
        with open(file, 'r', encoding = 'utf-16-be') as f:
            lines = [line.rstrip() for line in f]   
    text_list = []
    xmin_list = []
    xmax_list = []
    interval_list = []
    for line in lines[14:]:  #informations needed begin on the 9th lines
        if 'text =' in line:
            line = line.split('"')[1]
            if line != "":
                text_list.append(line)
                #print (len(text_list), len(xmin_list), len(xmax_list), len(interval_list), line, interval_list[-2:])
            else:
                xmin_list = xmin_list[:-1]
                xmax_list = xmax_list[:-1]
                interval_list = interval_list[:-1]
        if 'xmin' in line:
            time_min = line.split('=')[1].strip(' ')
            xmin_list.append(time_min)
        if 'xmax' in line:
            time_max = line.split('=')[1].strip(' ')
            xmax_list.append(time_max)
        if 'intervals [' in line:
            interval = line.split('[')[1].strip(']:')
            interval_list.append(interval)
        if 'intervals: size' in line:
            xmin_list = xmin_list[:-1]
            xmax_list = xmax_list[:-1]
    xmin_list = [float(item) for item in xmin_list]
    xmax_list = [float(item) for item in xmax_list]
    interval_list = [int(item) for item in interval_list]

    df = pd.DataFrame()
    df['text'] = text_list
    df['xmin'] = xmin_list
    df['xmax'] = xmax_list
    df['interval'] = interval_list
    df['tier'] = get_interval(df['interval'].tolist())
    df['start'] = df['xmin'].apply(lambda x: int(np.floor(x*16000)))
    df['end'] = df['xmax'].apply(lambda x: int(np.ceil(x*16000)))
    return df

def get_interval(time_list):
    interval_all = []
    interval_curr = 1
    for i in range(len(time_list)):
        if i == 0:
            interval_all.append(1)
        else:
            if time_list[i] > time_list[i - 1]:
                interval_all.append(interval_curr)
                continue
            else:
                interval_curr = interval_curr + 1
                interval_all.append(interval_curr)
    return interval_all



In [9]:
#textgrid_dir = '/work/jh2lab/elic_model/cr_audio_data' + '/ckm009-2023-05-26-Žminj/ckm009-2023-05-26-Žminj_01.TextGrid'
#textgrid_dir = '/work/jh2lab/elic_model/cr_audio_data' + '/ckm008-2023-05-29-Crikvenica/ckm008-2023-05-29-Crikvenica_01.TextGrid'
textgrid_dir = '/work/jh2lab/elic_model/cr_audio_data' + '/ckm015-Klana/ckm015-2023-06-08-Klana_01.TextGrid'
df = get_audio_df(textgrid_dir)
df

,text,xmin,xmax,interval,tier,start,end
0,Znači možemo onda krenuti,0.000000,1.683446,1,1,0,26936
1,((breath)),1.683446,2.060916,2,1,26935,32975
2,"od mjesta gdje ste rođeni, gdje ste odrasli, V...",2.060916,7.407497,3,1,32974,118520
3,x,8.185315,8.375956,5,1,130965,134016
4,"Ja, klanjski.",11.613783,12.403039,7,1,185820,198449
...,...,...,...,...,...,...,...
222,"A na Viliju Božju se je isto, ovi,",560.602273,563.151690,222,2,8969636,9010428
223,"@ bila je kot večera, na šest večer, ovi, je s...",563.151690,568.408864,223,2,9010427,9094542
224,((breath)),568.408864,569.095173,224,2,9094541,9105523
225,"Tu, ovi, se je kuhalo (.) makaroni, tu (@-) pa...",569.095173,574.689761,225,2,9105522,9195037


In [10]:
df_12 = df[df['tier'].isin([1,2])]
df_12

,text,xmin,xmax,interval,tier,start,end
0,Znači možemo onda krenuti,0.000000,1.683446,1,1,0,26936
1,((breath)),1.683446,2.060916,2,1,26935,32975
2,"od mjesta gdje ste rođeni, gdje ste odrasli, V...",2.060916,7.407497,3,1,32974,118520
3,x,8.185315,8.375956,5,1,130965,134016
4,"Ja, klanjski.",11.613783,12.403039,7,1,185820,198449
...,...,...,...,...,...,...,...
222,"A na Viliju Božju se je isto, ovi,",560.602273,563.151690,222,2,8969636,9010428
223,"@ bila je kot večera, na šest večer, ovi, je s...",563.151690,568.408864,223,2,9010427,9094542
224,((breath)),568.408864,569.095173,224,2,9094541,9105523
225,"Tu, ovi, se je kuhalo (.) makaroni, tu (@-) pa...",569.095173,574.689761,225,2,9105522,9195037


In [11]:
df_12.head(30)

,text,xmin,xmax,interval,tier,start,end
0,Znači možemo onda krenuti,0.000000,1.683446,1,1,0,26936
1,((breath)),1.683446,2.060916,2,1,26935,32975
2,"od mjesta gdje ste rođeni, gdje ste odrasli, V...",2.060916,7.407497,3,1,32974,118520
3,x,8.185315,8.375956,5,1,130965,134016
4,"Ja, klanjski.",11.613783,12.403039,7,1,185820,198449
5,Mhm.,31.772204,32.317438,9,1,508355,517080
6,Dobro.,34.094261,34.616618,11,1,545508,553866
7,((breath)),34.616618,35.066532,12,1,553865,561065
8,"Ah, sjećate se čega @ iz djetinjstva posebno?",35.066532,38.436010,13,1,561064,614977
9,"Kako ste se igrali, kako ste se družili?",38.706721,41.174183,15,1,619307,658787


In [12]:
df_12['length'] = df_12['xmax'] - df_12['xmin']
df_12

,text,xmin,xmax,interval,tier,start,end,length
0,Znači možemo onda krenuti,0.000000,1.683446,1,1,0,26936,1.683446
1,((breath)),1.683446,2.060916,2,1,26935,32975,0.377470
2,"od mjesta gdje ste rođeni, gdje ste odrasli, V...",2.060916,7.407497,3,1,32974,118520,5.346581
3,x,8.185315,8.375956,5,1,130965,134016,0.190642
4,"Ja, klanjski.",11.613783,12.403039,7,1,185820,198449,0.789256
...,...,...,...,...,...,...,...,...
222,"A na Viliju Božju se je isto, ovi,",560.602273,563.151690,222,2,8969636,9010428,2.549417
223,"@ bila je kot večera, na šest večer, ovi, je s...",563.151690,568.408864,223,2,9010427,9094542,5.257174
224,((breath)),568.408864,569.095173,224,2,9094541,9105523,0.686309
225,"Tu, ovi, se je kuhalo (.) makaroni, tu (@-) pa...",569.095173,574.689761,225,2,9105522,9195037,5.594588


In [13]:
import numpy as np
np.max(df_12.length)

10.90483917016985

In [14]:
text = " ".join(df_12['text'].tolist())
text

'Znači možemo onda krenuti ((breath)) od mjesta gdje ste rođeni, gdje ste odrasli, Vaša sjećanja na djetinjstvo, igara. x Ja, klanjski. Mhm. Dobro. ((breath)) Ah, sjećate se čega @ iz djetinjstva posebno? Kako ste se igrali, kako ste se družili? (Ma) kako ste se vi igrali, na ča? Mhm. Ča je to? Mhm. Mhm. Dobro, a kad ste bili već divojka, kako je onda bilo? Mhm. I kako su se @, obično ste se zaljubljivali u dečke iz istog mjesta ili iz drugih, kako je to bilo onda? Mhm. ((breath)) A što Vam se čini @ taj način kako ste Vi odrastali, kako @ kad usporedite sa današnjim vremenom, ima li neke razlike? Mhm. A ste imali braće i sestre? (x) ste rekli (kujina) Su Vas oni čuvali? Mhm. (x) Tako su Vas i pustili s njimi, jer inače možda i ne bi? ((laughs)) Mhm. Mhm. ((breath)) A @- kako ste bili ko @ dijete? Mirna? Tako su Vas i naučili, (jel) da budete fini pa onda. E, mhm, mhm. ((breath)) A šta Vam se čini, ovaj. A kako s- s- s- se išlo obavezno nedjeljom na misu, ili šta, kako je to bilo nedje

In [21]:
def locate_end(xmin_list, xmax_list, i):
    start_curr = xmin_list[i]
    if xmax_list[-1] - start_curr <= 20: # if the end of max < 15, return the last xmax
        return xmax_list[-1], len(xmin_list) + 1
    if xmin_list[i + 1] - start_curr > 30: # if next start leaves too long of blank
        return start_curr + 20, i + 1
    while i < len(xmin_list):
        end_curr = xmax_list[i]
        #print ('----', end_curr, i)
        if end_curr - start_curr >= 20:
            if end_curr - start_curr <= 25:
                return end_curr, i + 1
            else:
                return start_curr + (25 - (xmax_list[i-1] - start_curr)), i 
        else:
            i = i + 1
            continue
    

In [15]:
def locate_end_small(xmin_list, xmax_list, i):
    start_curr = xmin_list[i]
    end_curr = xmax_list[i]

    if xmax_list[-1] - start_curr <= 10: # if the end of max < 15, return the last xmax
        return xmax_list[-1], len(xmin_list) + 1

    if end_curr - start_curr >= 10:
        return end_curr, i + 1
    
    while i < len(xmin_list):
        end_curr = xmax_list[i]
        #print ('----', start_curr, end_curr, i)
        if end_curr - start_curr >= 10:
            
            if xmin_list[i] - start_curr >= 10: # current start - last end > 10 blank, sep the blank and extend the last one
                return start_curr + 8,i
            else:
                return end_curr, i + 1
      
        else:
            i = i + 1
            continue

In [16]:
# chunk size 15 - 20 seconds

#file_name = 'ckm008-2023-05-29-Crikvenica_01'
#file_name = 'ckm009-2023-05-26-Žminj_01'
file_name = 'ckm015-2023-06-08-Klana_01'

start = []
end = []
start_id = []
end_id = []
file_info = []
chunk_name = []

#for file_name in file_tier_list:
#for file_name in [file_names[0]]:
for tier in [1,2]:
    print (tier)

    df_curr = df_12[df_12['tier'] == tier]
    xmin_list = df_curr['xmin'].tolist()
    xmax_list = df_curr['xmax'].tolist()
    
    i = 0
    j = 0 # counting the file chunk number
    while i < len(xmin_list):
        start_curr = xmin_list[i]
        i_curr = i
        # locate the one that > 15 and  < 25 in endlist
        end_curr = locate_end_small(xmin_list, xmax_list, i)[0]
        i = locate_end_small(xmin_list, xmax_list, i)[1]
        print (start_curr, end_curr, i_curr, i, end_curr-start_curr)
        
        j = j + 1 # chunk number update
        
        start.append(start_curr)
        end.append(end_curr)
        start_id.append(i_curr)
        end_id.append(i)
        file_info.append(tier)
        chunk_name.append(file_name + '_' + str(j))

1
0.0 8.0 0 4 8.0
11.613783194980117 19.613783194980115 4 5 7.999999999999998
31.772203774158278 39.77220377415828 5 10 8.0
78.2307955121999 86.2307955121999 10 13 8.0
112.05124969742033 120.05124969742033 13 14 8.0
152.29482381081965 160.29482381081965 14 16 8.0
213.4958215541538 221.4958215541538 16 18 8.0
232.56175831657816 240.56175831657816 18 21 8.0
270.9776229232727 278.9776229232727 21 24 8.0
286.22039852139994 296.9381499245994 24 28 10.71775140319943
296.9381499245994 308.7166885713972 28 33 11.778538646797813
339.8353854922216 354.456540218196 33 38 14.621154725974407
396.7811603558644 404.7811603558644 38 43 8.0
407.6191670446722 415.6191670446722 43 45 8.0
459.98941607407556 467.98941607407556 45 46 8.0
493.4991461849065 501.4991461849065 46 47 8.0
553.1048235419587 561.1048235419587 47 48 8.0
578.4791949402002 579.0587451371501 48 50 0.579550196949981
2
7.407497362516724 18.69572028591084 0 6 11.288222923394116
18.69572028591084 31.287326620639067 6 11 12.591606334728226


In [17]:
df_chunk = pd.DataFrame()

df_chunk['chunk_name'] = chunk_name
df_chunk['file_tier'] = file_info
df_chunk['start'] = start
df_chunk['end'] = end
df_chunk['start_id'] = start_id
df_chunk['end_id'] = end_id
df_chunk

,chunk_name,file_tier,start,end,start_id,end_id
0,ckm015-2023-06-08-Klana_01_1,1,0.000000,8.000000,0,4
1,ckm015-2023-06-08-Klana_01_2,1,11.613783,19.613783,4,5
2,ckm015-2023-06-08-Klana_01_3,1,31.772204,39.772204,5,10
3,ckm015-2023-06-08-Klana_01_4,1,78.230796,86.230796,10,13
4,ckm015-2023-06-08-Klana_01_5,1,112.051250,120.051250,13,14
...,...,...,...,...,...,...
56,ckm015-2023-06-08-Klana_01_39,2,521.370247,531.628617,161,164
57,ckm015-2023-06-08-Klana_01_40,2,531.628617,542.592965,164,168
58,ckm015-2023-06-08-Klana_01_41,2,542.592965,552.811236,168,172
59,ckm015-2023-06-08-Klana_01_42,2,556.178483,568.408864,172,175


In [13]:
df_chunk.head(30)

,chunk_name,file_tier,start,end,start_id,end_id
0,ckm015-2023-06-08-Klana_01_1,1,0.000000,8.000000,0,4
1,ckm015-2023-06-08-Klana_01_2,1,11.613783,19.613783,4,5
2,ckm015-2023-06-08-Klana_01_3,1,31.772204,39.772204,5,10
3,ckm015-2023-06-08-Klana_01_4,1,78.230796,86.230796,10,13
4,ckm015-2023-06-08-Klana_01_5,1,112.051250,120.051250,13,14
5,ckm015-2023-06-08-Klana_01_6,1,152.294824,160.294824,14,16
6,ckm015-2023-06-08-Klana_01_7,1,213.495822,221.495822,16,18
7,ckm015-2023-06-08-Klana_01_8,1,232.561758,240.561758,18,21
8,ckm015-2023-06-08-Klana_01_9,1,270.977623,278.977623,21,24
9,ckm015-2023-06-08-Klana_01_10,1,286.220399,296.938150,24,28


In [14]:
df_12.head(30)

,text,xmin,xmax,interval,tier,start,end,length
0,Znači možemo onda krenuti,0.000000,1.683446,1,1,0,26936,1.683446
1,((breath)),1.683446,2.060916,2,1,26935,32975,0.377470
2,"od mjesta gdje ste rođeni, gdje ste odrasli, V...",2.060916,7.407497,3,1,32974,118520,5.346581
3,x,8.185315,8.375956,5,1,130965,134016,0.190642
4,"Ja, klanjski.",11.613783,12.403039,7,1,185820,198449,0.789256
5,Mhm.,31.772204,32.317438,9,1,508355,517080,0.545235
6,Dobro.,34.094261,34.616618,11,1,545508,553866,0.522358
7,((breath)),34.616618,35.066532,12,1,553865,561065,0.449914
8,"Ah, sjećate se čega @ iz djetinjstva posebno?",35.066532,38.436010,13,1,561064,614977,3.369478
9,"Kako ste se igrali, kako ste se družili?",38.706721,41.174183,15,1,619307,658787,2.467463


In [18]:

def get_chunk_text(start_id, end_id, file_tier):
    df_curr = df[df['tier'] == file_tier] 
    text_list = df_curr['text'].tolist()
    text_chunks = text_list[start_id : end_id]
    text = ' '.join(text_chunks)
    return text

get_chunk_text(0, 9, 1)

'Znači možemo onda krenuti ((breath)) od mjesta gdje ste rođeni, gdje ste odrasli, Vaša sjećanja na djetinjstvo, igara. x Ja, klanjski. Mhm. Dobro. ((breath)) Ah, sjećate se čega @ iz djetinjstva posebno?'

In [19]:
len(df_chunk)

61

In [20]:
df_chunk['text'] = df_chunk.apply(lambda x: get_chunk_text(x.start_id, x.end_id, x.file_tier), axis = 1)
df_chunk

,chunk_name,file_tier,start,end,start_id,end_id,text
0,ckm015-2023-06-08-Klana_01_1,1,0.000000,8.000000,0,4,Znači možemo onda krenuti ((breath)) od mjesta...
1,ckm015-2023-06-08-Klana_01_2,1,11.613783,19.613783,4,5,"Ja, klanjski."
2,ckm015-2023-06-08-Klana_01_3,1,31.772204,39.772204,5,10,"Mhm. Dobro. ((breath)) Ah, sjećate se čega @ i..."
3,ckm015-2023-06-08-Klana_01_4,1,78.230796,86.230796,10,13,"(Ma) kako ste se vi igrali, na ča? Mhm. Ča je to?"
4,ckm015-2023-06-08-Klana_01_5,1,112.051250,120.051250,13,14,Mhm.
...,...,...,...,...,...,...,...
56,ckm015-2023-06-08-Klana_01_39,2,521.370247,531.628617,161,164,"((click)) bile su jako delikate, o- ovi, onako..."
57,ckm015-2023-06-08-Klana_01_40,2,531.628617,542.592965,164,168,"I onda smo sili na škambljiće tako, ovi. ((sig..."
58,ckm015-2023-06-08-Klana_01_41,2,542.592965,552.811236,168,172,((breath)) To mi je va sjećanju od Božića. Mis...
59,ckm015-2023-06-08-Klana_01_42,2,556.178483,568.408864,172,175,Kaj se je još osim bakalara? Bakalar s p- n- i...


In [21]:
import librosa
#audio_dir = '/work/jh2lab/elic_model/cr_audio_data' + '/ckm009-2023-05-26-Žminj/ckm009-2023-05-26-Žminj_01.wav'
#audio_dir = '/work/jh2lab/elic_model/cr_audio_data' + '/ckm008-2023-05-29-Crikvenica/ckm008-2023-05-29-Crikvenica_01.wav'
audio_dir = '/work/jh2lab/elic_model/cr_audio_data' + '/ckm015-Klana/ckm015-2023-06-08-Klana_01.wav'

speech, sample_rate = librosa.load(audio_dir, sr=16000)

In [22]:
len(speech)/16000

579.5984375

In [23]:
# load model
from transformers import pipeline
pipe_1 = pipeline(model="ninninz/whisper-large-v3-croarian_10") 

2024-03-25 20:25:02.859423: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 20:25:02.859493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 20:25:03.123558: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 20:25:03.840650: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 20:25:12.836326: W tensorflow/comp

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
def get_whisper_text_1(t1, t2, speech):
    t1 = int(t1 * 16000) #Works in milliseconds
    t2 = int(t2 * 16000)
    newAudio = speech[t1:t2]
    result = pipe_1(newAudio)['text']
    return result

get_whisper_text_1(0,10, speech)

'Či možemo onda krenuti od mjesta gdje ste rođeni, gdje ste odrasli, vaši sječan na djetinstvo i ga gra.'

In [25]:
pipe_2 = pipeline(model="ninninz/whisper-large-v3-croarian_20") 

def get_whisper_text_2(t1, t2, speech):
    t1 = int(t1 * 16000) #Works in milliseconds
    t2 = int(t2 * 16000)
    newAudio = speech[t1:t2]
    result = pipe_2(newAudio)['text']
    return result

get_whisper_text_2(0,10, speech)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'Znači možemo onda krenuti od mjesta gdje ste rođeni, gdje ste odrasli, va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va va'

In [26]:
pipe_3 = pipeline(model="ninninz/whisper-large-v3-croarian_30") 

def get_whisper_text_3(t1, t2, speech):
    t1 = int(t1 * 16000) #Works in milliseconds
    t2 = int(t2 * 16000)
    newAudio = speech[t1:t2]
    result = pipe_3(newAudio)['text']
    return result

get_whisper_text_3(10,20, speech)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'Klanski ću govorit.'

In [27]:
pipe_4 = pipeline(model="ninninz/whisper-large-v3-croarian_overlap_removed_10") 

def get_whisper_text_4(t1, t2, speech):
    t1 = int(t1 * 16000) #Works in milliseconds
    t2 = int(t2 * 16000)
    newAudio = speech[t1:t2]
    result = pipe_4(newAudio)['text']
    return result

get_whisper_text_4(10,20, speech)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'Klanski.'

In [28]:
pipe_5 = pipeline(model="ninninz/whisper-large-v3-croarian_overlap_removed_20") 

def get_whisper_text_5(t1, t2, speech):
    t1 = int(t1 * 16000) #Works in milliseconds
    t2 = int(t2 * 16000)
    newAudio = speech[t1:t2]
    result = pipe_5(newAudio)['text']
    return result

get_whisper_text_5(10,20, speech)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'Kako je on?'

In [29]:
pipe_6 = pipeline(model="ninninz/whisper-large-v3-croarian_overlap_removed_30") 

def get_whisper_text_6(t1, t2, speech):
    t1 = int(t1 * 16000) #Works in milliseconds
    t2 = int(t2 * 16000)
    newAudio = speech[t1:t2]
    result = pipe_6(newAudio)['text']
    return result

get_whisper_text_6(10,20, speech)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'Kalj i Makeda.'

In [30]:
pipe_7 = pipeline(model="openai/whisper-large-v3") 

def get_whisper_text_7(t1, t2, speech):
    t1 = int(t1 * 16000) #Works in milliseconds
    t2 = int(t2 * 16000)
    newAudio = speech[t1:t2]
    result = pipe_7(newAudio, generate_kwargs={"language": "croatian"})['text']
    return result

get_whisper_text_7(0,10, speech)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


' Možemo onda krenuti od mjesta gdje ste rođeni, gdje ste odrasli, vaše sjećanje na djetinstvo, igara.'

In [31]:
df_chunk['transcription_v1'] = df_chunk.apply(lambda x: get_whisper_text_1(x.start, x.end, speech), axis = 1)
df_chunk

,chunk_name,file_tier,start,end,start_id,end_id,text,transcription_v1
0,ckm015-2023-06-08-Klana_01_1,1,0.000000,8.000000,0,4,Znači možemo onda krenuti ((breath)) od mjesta...,Či možemo onda krenuti od mjesta gdje ste rođe...
1,ckm015-2023-06-08-Klana_01_2,1,11.613783,19.613783,4,5,"Ja, klanjski.","Ja, pajnski."
2,ckm015-2023-06-08-Klana_01_3,1,31.772204,39.772204,5,10,"Mhm. Dobro. ((breath)) Ah, sjećate se čega @ i...",Mhm. Dobro. A sjećate se čega eh iz djetinjstv...
3,ckm015-2023-06-08-Klana_01_4,1,78.230796,86.230796,10,13,"(Ma) kako ste se vi igrali, na ča? Mhm. Ča je to?",Ma kako ste se igrali? Na ča?
4,ckm015-2023-06-08-Klana_01_5,1,112.051250,120.051250,13,14,Mhm.,Mhm.
...,...,...,...,...,...,...,...,...
56,ckm015-2023-06-08-Klana_01_39,2,521.370247,531.628617,161,164,"((click)) bile su jako delikate, o- ovi, onako...","Bile su jako delikate ovi, onako za razbit zmr..."
57,ckm015-2023-06-08-Klana_01_40,2,531.628617,542.592965,164,168,"I onda smo sili na škambljiće tako, ovi. ((sig...",I onda smo sili na škambljiće tako ovi. I onda...
58,ckm015-2023-06-08-Klana_01_41,2,542.592965,552.811236,168,172,((breath)) To mi je va sjećanju od Božića. Mis...,Da.
59,ckm015-2023-06-08-Klana_01_42,2,556.178483,568.408864,172,175,Kaj se je još osim bakalara? Bakalar s p- n- i...,Kaj se još osin bakalara. Bakalar i palenta se...


In [ ]:
df_chunk['transcription_v2'] = df_chunk.apply(lambda x: get_whisper_text_2(x.start, x.end, speech), axis = 1)
df_chunk

In [ ]:
df_chunk['transcription_v3'] = df_chunk.apply(lambda x: get_whisper_text_3(x.start, x.end, speech), axis = 1)
df_chunk

In [ ]:
df_chunk['transcription_v4'] = df_chunk.apply(lambda x: get_whisper_text_4(x.start, x.end, speech), axis = 1)
df_chunk

In [ ]:
df_chunk['transcription_v5'] = df_chunk.apply(lambda x: get_whisper_text_5(x.start, x.end, speech), axis = 1)
df_chunk

In [ ]:
df_chunk['transcription_v6'] = df_chunk.apply(lambda x: get_whisper_text_6(x.start, x.end, speech), axis = 1)
df_chunk

In [ ]:
df_chunk['transcription_original'] = df_chunk.apply(lambda x: get_whisper_text_7(x.start, x.end, speech), axis = 1)
df_chunk

In [ ]:
df_chunk

In [ ]:
#df_chunk.to_csv('/home/sz38235/workDir/elic/ckm009-2023-05-26-Žminj_01_m6.csv')
#df_chunk.to_csv('/home/sz38235/workDir/elic/ckm008-2023-05-29-Crikvenica_01_m6.csv')
df_chunk.to_csv('/home/sz38235/workDir/elic/ckm015-2023-06-08-Klana_01_m6.csv')

# test v3

In [1]:
import pandas as pd
df = pd.read_csv('/home/sz38235/workDir/elic/ckm008-2023-05-29-Crikvenica/ckm008-2023-05-29-Crikvenica_01_3.csv')
df

,Unnamed: 0,chunk_name,file_tier,start,end,start_id,end_id,transcription,transcription_original
0,0,ckm008-2023-05-29-Crikvenica_01_1,1,0.000000,6.037918,0,1,"Ja, ne govorin ovdjanskon čakvicun, ali ću nas...","Ja ne govorim lovljanskom čakovicom, ali ću n..."
1,1,ckm008-2023-05-29-Crikvenica_01_2,1,6.363340,11.544583,1,2,Evo da vas pitan prvo kade ste rođeni i kade s...,Evo da vas prvo pitan kada ste rodjeni i kada...
2,2,ckm008-2023-05-29-Crikvenica_01_3,1,35.144858,42.317897,2,4,"Mhm. Evo, to je ono, ukratko.","Evo, to je ono ukratko. A kako je bilo kad st..."
3,3,ckm008-2023-05-29-Crikvenica_01_4,1,42.537059,46.561299,4,5,"va kakovo ste bili familiji, kulko vas je bilo...",A kakvo ste bili familije i koliko vas je bilo?
4,4,ckm008-2023-05-29-Crikvenica_01_5,1,51.120000,56.120000,5,6,"A dal' je bilo još koga va kuće, recimo?",A da li je bilo još koga u vašoj kući?
...,...,...,...,...,...,...,...,...,...
85,85,ckm008-2023-05-29-Crikvenica_01_66,2,526.800000,531.800000,87,88,Prišla san va Novi. A drugi dio me zanima. Na ...,Prišla sam vanovi. A drugi dio me zanima.
86,86,ckm008-2023-05-29-Crikvenica_01_67,2,534.784679,540.255729,88,91,"e- ovako, mi va novon. Kad rečemo novi, onda s...","Ovako, mi imamo ono. Kad rećemo novi onda se ..."
87,87,ckm008-2023-05-29-Crikvenica_01_68,2,540.574510,544.715483,91,93,I va općen govoru. O- ovo nećete nikad čut. To...,I vao će govor. Ovo nećete neka čuti.
88,88,ckm008-2023-05-29-Crikvenica_01_69,2,544.987775,553.400000,93,95,"Recimo, ljudi ki su va Zagrebu, novljani gredu...","Recimo, ljudi koji su u Zagrebu, novljani, gr..."


In [2]:
# load model
from transformers import pipeline
pipe = pipeline(model="ninninz/whisper-large-v3-croatian-v3") 

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [3]:
import librosa
#audio_dir = '/home/sz38235/workDir/elic/ckm009-2023-05-26-Žminj/ckm009-2023-05-26-Žminj_01.wav'

audio_dir = '/home/sz38235/workDir/elic/ckm008-2023-05-29-Crikvenica/ckm008-2023-05-29-Crikvenica_01.wav'
speech, sample_rate = librosa.load(audio_dir, sr=16000)

In [4]:
len(speech)/16000

559.6298125

In [6]:
def get_whisper_text(t1, t2, speech):
    t1 = int(t1 * 16000) #Works in milliseconds
    t2 = int(t2 * 16000)
    newAudio = speech[t1:t2]
    result = pipe(newAudio)['text']
    return result

get_whisper_text(0,10, speech)

'Ja ne govorin novdjanskon čakicun, ali ću nastojat na svoje pa da.).laughing). Evo da vas pitan.'

In [8]:
get_whisper_text(20,30, speech)

'A za novo, posebeno! I za se čini!'

In [9]:
df['transcription_v3'] = df.apply(lambda x: get_whisper_text(x.start, x.end, speech), axis = 1)
df

,Unnamed: 0,chunk_name,file_tier,start,end,start_id,end_id,transcription,transcription_original,transcription_v3
0,0,ckm008-2023-05-29-Crikvenica_01_1,1,0.000000,6.037918,0,1,"Ja, ne govorin ovdjanskon čakvicun, ali ću nas...","Ja ne govorim lovljanskom čakovicom, ali ću n...","Ja ne govorin novdjanjskon čakicon, ali ću nas..."
1,1,ckm008-2023-05-29-Crikvenica_01_2,1,6.363340,11.544583,1,2,Evo da vas pitan prvo kade ste rođeni i kade s...,Evo da vas prvo pitan kada ste rodjeni i kada...,Evo da vas pitan prvo kade ste rođeni i kade s...
2,2,ckm008-2023-05-29-Crikvenica_01_3,1,35.144858,42.317897,2,4,"Mhm. Evo, to je ono, ukratko.","Evo, to je ono ukratko. A kako je bilo kad st...","M� Evo, to je ono u kratko."
3,3,ckm008-2023-05-29-Crikvenica_01_4,1,42.537059,46.561299,4,5,"va kakovo ste bili familiji, kulko vas je bilo...",A kakvo ste bili familije i koliko vas je bilo?,"Va kakvoj ste bili familiji, kulko vas je bilo..."
4,4,ckm008-2023-05-29-Crikvenica_01_5,1,51.120000,56.120000,5,6,"A dal' je bilo još koga va kuće, recimo?",A da li je bilo još koga u vašoj kući?,"A dal' je bilo još koga va kuće, recimo?"
...,...,...,...,...,...,...,...,...,...,...
85,85,ckm008-2023-05-29-Crikvenica_01_66,2,526.800000,531.800000,87,88,Prišla san va Novi. A drugi dio me zanima. Na ...,Prišla sam vanovi. A drugi dio me zanima.,Prišla san va Novi.
86,86,ckm008-2023-05-29-Crikvenica_01_67,2,534.784679,540.255729,88,91,"e- ovako, mi va novon. Kad rečemo novi, onda s...","Ovako, mi imamo ono. Kad rećemo novi onda se ...","E ovako, mi van u. Kad rečemo novi, onda se to..."
87,87,ckm008-2023-05-29-Crikvenica_01_68,2,540.574510,544.715483,91,93,I va općen govoru. O- ovo nećete nikad čut. To...,I vao će govor. Ovo nećete neka čuti.,"I va općen govoru ovo nećete nika čut,"
88,88,ckm008-2023-05-29-Crikvenica_01_69,2,544.987775,553.400000,93,95,"Recimo, ljudi ki su va Zagrebu, novljani gredu...","Recimo, ljudi koji su u Zagrebu, novljani, gr...","Recimo, ljudi ki su va Zagrebu, novljani gredu..."


In [10]:
df.to_csv('/home/sz38235/workDir/elic/ckm008-2023-05-29-Crikvenica/ckm008-2023-05-29-Crikvenica_01_3.csv')
